In [17]:
import pandas as pd
import re
import numpy as np

In [20]:
data.head()

,Link_house,user,user_link,time,Tinh/TP,Quan/Huyen,Diện tích:,Địa chỉ:,price,price_title,...,Phòng ngủ,Có sân,Địa điểm,Hướng nhà:,Kinh doanh,Hướng ban công:,An ninh,Phòng WC,Dân trí,Gần bệnh viện
0,https://batdongsan.vn/can-ban-nha-kien-hung-ha...,Dương Nguyễn,https://batdongsan.vn/duongnguyen.181874,"[<li><i class=""uk-icon-angle-right""></i><span>...",Hà Nội,Hà Đông,35 m,"Quận Hà Đông, Hà Nội",3.2,0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://batdongsan.vn/nha-moi-sai-dong-long-bi...,Bùi Mạnh Cương,https://batdongsan.vn/buimanhcuong.189440,"[<li><i class=""uk-icon-angle-right""></i><span>...",Hà Nội,Long Biên,NaN,Long Biên,5.2,5.NHỈNH,...,4.0,có sân,"tỉnh như đường 5B, đường Nguyễn Văn Linh",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://batdongsan.vn/ban-gap-nha-pho-truong-c...,lưu văn anh,https://batdongsan.vn/luuvananh.192740,"[<li><i class=""uk-icon-angle-right""></i><span>...",Hà Nội,Đống Đa,34 m,Hà Nội,3,3.0,...,4.0,sân bóng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://batdongsan.vn/nha-1tret-2lau-hem-xe-ho...,Hồ Ngọc Long,https://batdongsan.vn/hongoclong.52923,"[<li><i class=""uk-icon-angle-right""></i><span>...",TP Hồ Chí Minh,Thủ Đức,50 m,"Đường Số 2, Phường Trường Thọ, Quận Thủ Đức, T...",5.6,5.6,...,3.0,sân phơi,"đường số 2 Trường Thọ Thủ Đức, giá 5,6 tỷ\r\nV...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://batdongsan.vn/ban-nha-hxh-sat-duong-so...,Dương Trí Dũng,https://batdongsan.vn/duongtridung.178383,"[<li><i class=""uk-icon-angle-right""></i><span>...",TP Hồ Chí Minh,Gò Vấp,64 m,Đường Số 8,8.8,8.8,...,5.0,sân thượng,Vị trí: nhà HXH sát Đường Số 8 Phường 11 Gò Vấ...,Nam,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
data.isnull().sum()

Link_house            0
user                  0
user_link             0
time                  0
Tinh/TP               0
Quan/Huyen            0
Diện tích:          790
Địa chỉ:           4689
price               370
price_title        2762
Tầng               5948
giá                   0
Diện tích          7954
Số tầng               0
Số lầu                0
trệt                  0
Gần chợ            5958
Gần trường         5743
Phòng ngủ:         3673
Phòng WC:          4179
Phòng ngủ          4282
Có sân             5966
Địa điểm           3145
Hướng nhà:         8545
Kinh doanh         7199
Hướng ban công:    9122
An ninh            7432
Phòng WC           6892
Dân trí            7855
Gần bệnh viện      8329
dtype: int64

In [22]:
data['time'] = data['time'].str.extract(r'datetime="([^"]+)"')
data['Địa điểm'] = data['Địa điểm'].fillna('')
data['Địa chỉ:'] = data['Địa chỉ:'].fillna('')
gop = []

def remove_prefix_area(text):
    a = re.sub(r'^((vị trí:|địa điểm:|khu vục:)\s*)', '', text)
    matches = re.findall(r'(?<=,)(.*?)(?=,|$)', a)

# Lọc các đoạn văn bản để chỉ lấy các địa chỉ
    addresses = [match.strip() for match in matches if re.search(r'\b\d+\b', match) is None]
    address = ", ".join(addresses)
    return address
data['Địa điểm'] = data['Địa điểm'].apply(remove_prefix_area)
for index, row in data.iterrows():
    gop.append(row['Địa chỉ:'] if len(row['Địa chỉ:']) > len(row['Địa điểm']) else row['Địa điểm'])
data['address'] = gop
data['PN'] = data.apply(lambda row: str(row['Phòng ngủ:']) if pd.notnull(row['Phòng ngủ:']) else str(row['Phòng ngủ']), axis=1)
data['WC'] = data.apply(lambda row: str(row['Phòng WC:']) if pd.notnull(row['Phòng WC:']) else str(row['Phòng WC']), axis=1)

data['area'] = data.apply(lambda row: row['Diện tích:'] if pd.notnull(row['Diện tích:']) else row['Diện tích'], axis=1)
data['floor'] = data.apply(lambda row: str(row['Tầng']) if pd.notnull(row['Tầng']) else str(row["Số tầng"]), axis=1)
#data['price_'] = data.apply(lambda row: str(row['price']) if pd.notnull(row['price']) else str(row["giá"]), axis=1)

data['Gần trường'] = data['Gần trường'].str.contains('[a-zA-Z]', regex=True).fillna(False)
data['Kinh doanh'] = data['Kinh doanh'].str.contains('[a-zA-Z]', regex=True).fillna(False)
data['Gần chợ'] = data['Gần chợ'].str.contains('[a-zA-Z]', regex=True).fillna(False)
data['Gần bệnh viện'] = data['Gần bệnh viện'].str.contains('[a-zA-Z]', regex=True).fillna(False)
data['An ninh'] = data['An ninh'].str.contains('[a-zA-Z]', regex=True).fillna(False)
data['Có sân'] = data['Có sân'].str.contains('[a-zA-Z]', regex=True).fillna(False)
data['Dân trí'] = data['Dân trí'].str.contains('[a-zA-Z]', regex=True).fillna(False)

In [23]:
data['PN']=data['PN'].replace('nan', np.nan)
data['WC']=data['WC'].replace('nan', np.nan)
data['floor']=data['floor'].replace('nan', np.nan)
#data['price_'] = data['price_'].replace('nan', np.nan)

In [24]:
data.drop('Tầng', inplace=True, axis=1)
data.drop('Số tầng', inplace=True, axis =1)
data.drop('Số lầu', inplace=True, axis =1)
data.drop('trệt', inplace=True, axis =1)
data.drop('Diện tích:', inplace=True, axis=1)
data.drop('Diện tích', inplace=True, axis=1)
data.drop('Phòng ngủ:', inplace=True, axis=1)
data.drop('Phòng WC:', inplace=True, axis=1)
data.drop('Phòng ngủ', inplace=True, axis=1)
data.drop('Phòng WC', inplace=True, axis=1)
data.drop('Địa điểm', inplace=True, axis=1)
data.drop('Địa chỉ:', inplace=True, axis=1)
#data.drop('price', inplace=True, axis=1)
#data.drop('giá', inplace=True, axis=1)
data.drop('Hướng nhà:', inplace=True, axis=1)
data.drop('Hướng ban công:', inplace=True, axis=1)
data.drop('address', inplace=True, axis=1)

In [25]:
data['PN'] = data['PN'].str.replace(' PN', '')
data['WC'] = data['WC'].str.replace(' WC','')
data['area'] = data['area'].str.replace(' m','')
data['area'] = data['area'].fillna('')
data['area'] = data['area'].apply(lambda x: ("".join(re.findall(r'[\d.]*', x))))
data['area']=data['area'].replace('',np.nan)



In [26]:
data['area']=data['area'].replace(',', '.', regex=True)
data['area']=data['area'].astype(float)
#data['price_']=data['price_'].replace(',', '.', regex=True)
#data['price_']=data['price_'].astype(float)
data['PN']=data['PN'].astype(float)
data['WC']=data['WC'].astype(float)
data['floor']=data['floor'].astype(float)

#data = data[data['price_'] >= 0.3]
#data = data[data['area'] >= 10]

#data['price_'] = data['price_'].replace('0', np.nan)
#data.loc[(data['price_'] >= 1000000) & (data['price_'] <= 15000000), 'price_'] /= 1000000
#data = data.dropna(subset=['price_'])

data['floor'] = data['floor'].replace(0.0,np.nan)
data['floor'] = data['floor'].replace(np.nan, 1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9744 entries, 0 to 870
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Link_house     9744 non-null   object 
 1   user           9744 non-null   object 
 2   user_link      9744 non-null   object 
 3   time           9744 non-null   object 
 4   Tinh/TP        9744 non-null   object 
 5   Quan/Huyen     9744 non-null   object 
 6   price          9374 non-null   object 
 7   price_title    6982 non-null   object 
 8   giá            9744 non-null   object 
 9   Gần chợ        9744 non-null   bool   
 10  Gần trường     9744 non-null   bool   
 11  Có sân         9744 non-null   bool   
 12  Kinh doanh     9744 non-null   bool   
 13  An ninh        9744 non-null   bool   
 14  Dân trí        9744 non-null   bool   
 15  Gần bệnh viện  9744 non-null   bool   
 16  PN             7660 non-null   float64
 17  WC             6325 non-null   float64
 18  area     

In [27]:
data_temp=data[['Có sân','Dân trí','An ninh','Kinh doanh','Gần chợ','Gần trường','Gần bệnh viện']]
data['Tien_ich'] = data_temp.apply(lambda row: sum(row), axis=1)
data.drop('Có sân', inplace=True, axis=1)
data.drop('Dân trí', inplace=True, axis=1)
data.drop('An ninh', inplace=True, axis=1)
data.drop('Kinh doanh', inplace=True, axis=1)
data.drop('Gần chợ', inplace=True, axis=1)
data.drop('Gần trường', inplace=True, axis=1)
data.drop('Gần bệnh viện', inplace=True, axis=1)

In [28]:
data.drop('user', inplace=True, axis=1)
data.drop('user_link', inplace=True, axis=1)
#data.drop('time', inplace=True, axis=1)

In [29]:
data.head()


,Link_house,time,Tinh/TP,Quan/Huyen,price,price_title,giá,PN,WC,area,floor,Tien_ich
0,https://batdongsan.vn/can-ban-nha-kien-hung-ha...,2023-06-10 15:53:19,Hà Nội,Hà Đông,3.2,0.3,0.3,NaN,NaN,35.0,5.0,2
1,https://batdongsan.vn/nha-moi-sai-dong-long-bi...,2023-06-10 16:12:35,Hà Nội,Long Biên,5.2,5.NHỈNH,0,4.0,4.0,NaN,4.0,1
2,https://batdongsan.vn/ban-gap-nha-pho-truong-c...,2023-06-10 15:29:46,Hà Nội,Đống Đa,3,3.0,0,4.0,NaN,34.0,4.0,2
3,https://batdongsan.vn/nha-1tret-2lau-hem-xe-ho...,2023-06-10 10:02:18,TP Hồ Chí Minh,Thủ Đức,5.6,5.6,5.6.Vị,3.0,NaN,50.0,3.0,2
4,https://batdongsan.vn/ban-nha-hxh-sat-duong-so...,2023-06-10 10:22:43,TP Hồ Chí Minh,Gò Vấp,8.8,8.8,0,5.0,NaN,64.0,5.0,2


In [30]:
import re

# Khai báo regex pattern để lấy số từ chuỗi
pattern = r'(\d+(?:\.\d+)?)'

# Áp dụng regex vào cột "price_title" và thay thế các giá trị bằng số
data['price_title'] = data['price_title'].apply(lambda x: re.findall(pattern, str(x))[0] if re.findall(pattern, str(x)) else x)

data['giá'] = data['giá'].apply(lambda x: re.findall(pattern, str(x))[0] if re.findall(pattern, str(x)) else x)

In [31]:
data['giá']=data['giá'].apply(lambda x: float(str(x).replace(',','.')))
data['price']=data['price'].apply(lambda x: float(str(x).replace(',','.')))
data['price_title']=data['price_title'].apply(lambda x: float(str(x).replace(',','.')))

In [32]:
data['price'] = data['price'].replace(0.0, np.nan)
data.loc[(data['price'] >= 1000000) & (data['price'] <= 15000000), 'price'] /= 1000000
data['price_'] = data.apply(lambda row: (row['price']) if pd.notnull(row['price']) else ((row["giá"]) if pd.notnull(row['giá']) else (row["price_title"])), axis=1)

In [14]:
# import pandas as pd
# import numpy as np

# # Tạo DataFrame mẫu


# # Tạo cột 'price_' và sao chép dữ liệu từ cột 'price'
data.loc[(data['price'] >= 1000000) & (data['price'] <= 15000000), 'price'] /= 1000000
data['price_'] = data['price']

# Kiểm tra giá trị của cột 'price_' và thực hiện các điều kiện
for i, row in data.iterrows():
    if row['price_'] < 0.5:
        if max(row['giá'], row['price_title']) >=1:
            max_value = max(row['giá'], row['price_title'])
            data.at[i, 'price_'] = max_value
        else:
            data.at[i, 'price_'] = row['price']   
    elif 100 <= row['price_'] <= 1000:
        if 10 <= row['giá'] <= 100 and 10 <= row['price_title'] <= 100:
            data.at[i, 'price_'] = row['giá']
        else:
            if 1 <= row['giá'] <= 100:
                data.at[i, 'price_'] = row['giá']
            else: 
                if 1 <= row['price_title'] <= 100:
                    data.at[i, 'price_'] = row['price_title']
                else:
                    data.at[i, 'price_'] = row['price'] 
    elif 1000 <= row['price_'] <= 10000:
        if 1 <= row['giá'] <= 100 and 1 <= row['price_title'] <= 100:
            data.at[i, 'price_'] = row['giá']
        else:
            if 10 <= row['giá'] <= 100:
                data.at[i, 'price_'] = row['giá']
            else: 
                if 10 <= row['price_title'] <= 100:
                    data.at[i, 'price_'] = row['price_title']
                else:
                    data.at[i, 'price_'] = row['price']          
    elif row['price_'] > 10000:
        if 1 <= row['giá'] <= 500 and 1 <= row['price_title'] <= 500:
            data.at[i, 'price_'] = row['giá']
        elif 1 <= row['price_title'] <= 500:
            data.at[i, 'price_'] = row['price_title']
        elif 1 <= row['giá'] <= 500:
            data.at[i, 'price_'] = row['giá']
        else:
            data.at[i, 'price_'] = row['price'] 
    else:
        data.at[i, 'price_'] = row['price']     

# # In DataFrame kết quả



In [33]:
data['price_'] = data['price_'].replace(0.0, np.nan)

data = data.dropna(subset=['price_'])

In [34]:
data.drop('price', inplace=True, axis=1)
data.drop('giá', inplace=True, axis=1)
data.drop('price_title', inplace=True, axis=1)

In [35]:
data = data[data['price_'] >= 0.3]
data = data[data['area'] >= 10]
data = data[data['price_'] < 1000]

In [37]:
Q1 = data['area'].quantile(0.25)
Q3 = data['area'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Lọc và xóa các outlier
data = data[(data['area'] >= lower_bound) & (data['area'] <= upper_bound)]
# In dataframe sau khi lọc outlier
data.shape

(8297, 10)

In [38]:
# Xác định ngưỡng outlier sử dụng IQR
Q1 = data['price_'].quantile(0.25)
Q3 = data['price_'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Lọc và xóa các outlier
data = data[(data['price_'] >= lower_bound) & (data['price_'] <= upper_bound)]

In [36]:
data.drop('Link_house', inplace=True, axis=1)
data.drop('time', inplace=True, axis=1)
data.head()

,Tinh/TP,Quan/Huyen,PN,WC,area,floor,Tien_ich,price_
0,Hà Nội,Hà Đông,NaN,NaN,35.0,5.0,2,3.2
2,Hà Nội,Đống Đa,4.0,NaN,34.0,4.0,2,3.0
3,TP Hồ Chí Minh,Thủ Đức,3.0,NaN,50.0,3.0,2,5.6
4,TP Hồ Chí Minh,Gò Vấp,5.0,NaN,64.0,5.0,2,8.8
5,Đồng Nai,Biên Hòa,2.0,NaN,200.0,1.0,2,7.2


In [80]:
data.to_csv("D:\\NM_KDL\\practice\\Project final\\data\\pre_end_data_real_null_out.csv")

In [81]:
data.describe()

,PN,WC,area,floor,Tien_ich,price_
count,6979.000000,5842.000000,8632.000000,8632.000000,8632.000000,8632.000000
mean,3.737785,3.255221,96.329328,2.940686,2.030816,10.361766
std,3.690291,4.068420,1115.889965,1.753281,1.438815,39.902356
min,1.000000,0.000000,10.000000,1.000000,0.000000,0.300000
25%,2.000000,2.000000,44.000000,2.000000,1.000000,3.600000
50%,3.000000,3.000000,60.000000,3.000000,2.000000,5.350000
75%,4.000000,4.000000,85.000000,4.000000,3.000000,8.000000
max,111.000000,111.000000,100103.000000,30.000000,7.000000,950.000000


In [72]:
data_=pd.read_csv("D:\\NM_KDL\\practice\\Project final\\data\\pre_end_data_fake_null.csv")

In [73]:
data_

,Tinh/TP,Quan/Huyen,PN,WC,area,floor,Tien_ich,price_
0,Hà Nội,Hà Đông,NaN,NaN,35.0,5.0,2,3.3
1,Hà Nội,Đống Đa,4.0,NaN,34.0,4.0,2,2.5
2,TP Hồ Chí Minh,Thủ Đức,3.0,NaN,50.0,3.0,2,6.3
3,TP Hồ Chí Minh,Gò Vấp,5.0,NaN,64.0,5.0,2,7.0
4,Hà Nội,Hà Đông,2.0,NaN,39.0,4.0,3,5.7
...,...,...,...,...,...,...,...,...
7486,Hà Nội,Hoàng Mai,5.0,3.0,54.0,4.0,1,7.1
7487,Hà Nội,Đống Đa,NaN,NaN,50.0,5.0,3,6.8
7488,TP Hồ Chí Minh,Thủ Đức,NaN,NaN,100.0,1.0,1,4.8
7489,TP Hồ Chí Minh,Quận 12,3.0,2.0,52.0,2.0,2,2.4


In [74]:
data_.shape

(7491, 8)

In [75]:
cols = ['Tinh/TP', 'Quan/Huyen', 'PN', 'WC', 'area','floor', 'Tien_ich']

data3 = pd.merge(data_, data, on=cols, how='outer', suffixes=('', '_y'))

# Lấy giá trị của cột price_ từ data2 khi không có sự khớp
data3['price_'] = data3['price_'].combine_first(data3['price__y'])


# Xóa các cột không cần thiết
data3.drop(data3.filter(regex='_y$').columns.tolist(), axis=1, inplace=True)
data3

,Tinh/TP,Quan/Huyen,PN,WC,area,floor,Tien_ich,price_
0,Hà Nội,Hà Đông,NaN,NaN,35.0,5.0,2,3.30
1,Hà Nội,Đống Đa,4.0,NaN,34.0,4.0,2,2.50
2,TP Hồ Chí Minh,Thủ Đức,3.0,NaN,50.0,3.0,2,6.30
3,TP Hồ Chí Minh,Gò Vấp,5.0,NaN,64.0,5.0,2,7.00
4,Hà Nội,Hà Đông,2.0,NaN,39.0,4.0,3,5.70
...,...,...,...,...,...,...,...,...
13506,Hà Nội,Cầu Giấy,6.0,NaN,80.0,7.0,1,21.50
13507,Hà Nội,Cầu Giấy,6.0,6.0,121.0,11.0,0,120.00
13508,Hà Nội,Hà Đông,6.0,NaN,817.3,5.0,3,7.30
13509,TP Hồ Chí Minh,Quận 8,2.0,0.0,87.0,3.0,1,3.85


In [76]:
data3 = data3.drop_duplicates(keep='first')
data3

,Tinh/TP,Quan/Huyen,PN,WC,area,floor,Tien_ich,price_
0,Hà Nội,Hà Đông,NaN,NaN,35.0,5.0,2,3.30
1,Hà Nội,Đống Đa,4.0,NaN,34.0,4.0,2,2.50
2,TP Hồ Chí Minh,Thủ Đức,3.0,NaN,50.0,3.0,2,6.30
3,TP Hồ Chí Minh,Gò Vấp,5.0,NaN,64.0,5.0,2,7.00
4,Hà Nội,Hà Đông,2.0,NaN,39.0,4.0,3,5.70
...,...,...,...,...,...,...,...,...
13506,Hà Nội,Cầu Giấy,6.0,NaN,80.0,7.0,1,21.50
13507,Hà Nội,Cầu Giấy,6.0,6.0,121.0,11.0,0,120.00
13508,Hà Nội,Hà Đông,6.0,NaN,817.3,5.0,3,7.30
13509,TP Hồ Chí Minh,Quận 8,2.0,0.0,87.0,3.0,1,3.85


In [924]:

data['PN'] = data['PN'].replace(0, np.nan)
data['WC'] = data['WC'].replace(0, np.nan)
data = data.dropna(subset=['PN'])
data = data.dropna(subset=['WC'])
data.shape

(4525, 10)

In [78]:
data3.to_csv("D:\\NM_KDL\\practice\\Project final\\data\\pre_end_data_fake_null_out.csv", index=False)